**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,Model,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, AveragePooling2D,Reshape,BatchNormalization, Input, Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act has to mode of action, one during training and one during test. In train we want to use the learned policy to check its efficiency but we also want to allow the model to try and learn new policies thus under a certain probability treshold defined by $\epsilon$ we randomly output a new behaviour. In test we simply output the learned policy.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        # self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        # self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The $\textit{board}$ represents the environment as a grid where value $0.5$ stands for cheese, $0$ for a neutral cell and $-1$ for a poisonous cell. It defines the rewards in the different cells. The $\textit{position}$ defines where the agent is on the grid and the bounds of the grid. This grid is padded to take in account the fact that the mouse sees $5\times 5$ cells at once.
Note : there was a mistake in the code of self.position as there was twice the line :self.position\[-2:, :\] = -1 and no occurence of self.position\[:, -2:\] = -1

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        game_over = False

        win = 0
        lose = 0

        while not game_over:
        # The agent performs an action
            action = agent.act(state, train = False)
            prev_state = state
            state, reward, game_over = env.act(action)

        # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 5.0/18.0. Average score (-13.0)
Win/lose count 10.0/9.0. Average score (-6.0)
Win/lose count 7.0/16.0. Average score (-7.0)
Win/lose count 9.0/14.0. Average score (-6.5)
Win/lose count 11.0/10.0. Average score (-5.0)
Win/lose count 14.0/26.0. Average score (-6.166666666666667)
Win/lose count 13.0/11.0. Average score (-5.0)
Win/lose count 12.5/17.0. Average score (-4.9375)
Win/lose count 8.0/12.0. Average score (-4.833333333333333)
Win/lose count 10.0/17.0. Average score (-5.05)
Final score: -5.05


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




---------------
We have by definition 
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}
We simply have to split the sum into two parts as follows
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[r(s,a) + \sum_{t = 1}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]  \\
Q^\pi(s,a)=E_{p^{\pi}}[r(s,a)] + E_{p^{\pi}}[\sum_{t = 1}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]
\end{equation*}
And we use the markov property to get
\begin{equation*}
\gamma Q^\pi(s,a) = E_{(s',a')\sim p(.|s,a)}[\sum_{t = 1}^{T}\gamma^{t}r(s',a')]
\end{equation*}
therefore,
\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}
 
Thus, the optimal Q function maximizing the reward defined as:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}
when inserted in the previous result gives us
\begin{equation*}
Q^{*}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{*}(s',a')]
\end{equation*}
Also because this Q function is optimal we have $E_{(s',a')\sim p(.|s,a)}[Q^{*}(s',a')] = E_{(s')\sim p(.|s,a)}[ \max_{a'} Q^{*}(s',a')]$ and get
\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
 
We now have that the $Q_\pi$-function is a fixed point of the Bellman operator $T$: 

\begin{equation*}
[T_\pi Q](s,a)= r(s,a) + \gamma \sum_{s'\in S} p(s'|s,a) Q(s', \pi(s')) \\
T_\pi Q_\pi = Q_\pi
\end{equation*}

And the $Q^*$-function is a fixed point of the optimal Bellman operator $T_*$: 

\begin{equation*}
[T_* Q](s,a)= r(s,a) + \gamma \sum_{s'\in S} p(s'|s,a) max_{a' \in A} Q(s', a') \\
T_* Q^* = Q^*
\end{equation*}
 
To find the fixed point of the optimal Bellman operator we can look at the expected distance between $T_* Q$ and $Q$. 
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert [T_* Q](s,a, \theta) -Q(s,a,\theta)\Vert^{2} =E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            self.memory[:-1] = self.memory[1:]
            self.memory[-1] = m

    def random_access(self):
        return np.random.randint(len(self.memory), size=1)[0]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        predict_action = self.model.predict(s.reshape(1,5,5,self.n_state))
        return np.argmax(predict_action)
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ind = self.memory.random_access()
            input_states[i],next_state,action,reward,game_over_ = self.memory.memory[ind]
            target_q[i] = self.model.predict(input_states[i].reshape(1,5,5,self.n_state))
            
            if game_over_:
                target_q[i][action] = reward
            else:
                tmp = self.model.predict(next_state.reshape(1,5,5,self.n_state))
                target_q[i][action] = reward + self.discount * (np.amax(tmp))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        clip = 8
        target_q = np.clip(target_q, -clip, clip)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        input_layer = Input(shape=(5,5,self.n_state))
        x = Flatten()(input_layer)
        
        x = Dense(512)(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Dense(128)(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Dense(128)(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Dense(64)(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        output_layer = Dense(self.n_action)(x)
        model = Model(inputs=input_layer, outputs=output_layer)
        print(model.summary())
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train' + str(epochs_train-1 - (epochs_train-1)%10) + '.mp4'))

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5, 5, 2)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               26112     
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
batc

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        input_layer = Input(shape=(5,5,self.n_state))
        x = Conv2D(32, (2, 2))(input_layer)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Conv2D(32, (2, 2))(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Flatten()(x)
        x = Dense(64)(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        output_layer = Dense(self.n_action)(x)
        model = Model(inputs=input_layer, outputs=output_layer)
        
        print(model.summary())
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
# HTML(display_videos('cnn_train90.mp4'))
HTML(display_videos('cnn_train' + str(epochs_train-1 - (epochs_train-1)%10) + '.mp4'))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 5, 5, 2)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 32)          288       
_________________________________________________________________
batch_normalization_5 (Batch (None, 4, 4, 32)          128       
_________________________________________________________________
activation_5 (Activation)    (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 32)          4128      
_________________________________________________________________
batch_normalization_6 (Batch (None, 3, 3, 32)          128       
_________________________________________________________________
activation_6 (Activation)    (None, 3, 3, 32)          0         
__________

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

temperatures = [0.1,0.2,0.3,0.7,0.9]
for t in temperatures:
    print('\nTemperature : ' + str(t))
    env = Environment(grid_size=size, max_time=T,temperature=t)
    print('Test of the CNN')
    test(agent_cnn,env, 5,prefix='cnn_test')
    print('Test of the FC')
    test(agent_fc,env, 5,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 5, 5, 2)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 32)          288       
_________________________________________________________________
batch_normalization_8 (Batch (None, 4, 4, 32)          128       
_________________________________________________________________
activation_8 (Activation)    (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 32)          4128      
_________________________________________________________________
batch_normalization_9 (Batch (None, 3, 3, 32)          128       
_________________________________________________________________
activation_9 (Activation)    (None, 3, 3, 32)          0         
__________

In [16]:
HTML(display_videos('cnn_test0.mp4'))

In [17]:
HTML(display_videos('fc_test0.mp4'))

Obviously, the agent falls in a local minima which consists in switching between two cells which are empty, so essentially not playing in order to not lose after eating a piece of cheese.
Also, it seems to me that the network still has  troubles remembering and taking in consideration the previously explored bits of the map.


As expected, when we change the temperature, for high values the cheese is everywhere and the model has no difficulty winning, while with low temperatures, the cheese is rarer and the agent tend to achieve more equalities because it's not explorign far enough.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            
            # we add the decreasing factor on epsilon
            agent.set_epsilon(agent.epsilon * 0.9)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b

    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        # self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        
        game_over = self.t > self.max_time
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1
        if train:
            # in training we increase the value of the rewards (this is equivalent to tuning the learning rate)
            reward = reward + 5*self.board[self.x, self.y]
        else:
            reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        self.t = self.t + 1
        
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))
        self.malus_position = np.zeros((self.grid_size,self.grid_size))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore' + str(epochs_train-1 - (epochs_train-1)%10) + '.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 5, 5, 3)           0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 4, 32)          416       
_________________________________________________________________
batch_normalization_15 (Batc (None, 4, 4, 32)          128       
_________________________________________________________________
activation_15 (Activation)   (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 32)          4128      
_________________________________________________________________
batch_normalization_16 (Batc (None, 3, 3, 32)          128       
_________________________________________________________________
activation_16 (Activation)   (None, 3, 3, 32)          0         
__________

In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 23.0/1.0. Average score (22.0)
Win/lose count 20.0/1.0. Average score (20.5)
Win/lose count 28.0/1.0. Average score (22.666666666666668)
Win/lose count 23.0/1.0. Average score (22.5)
Win/lose count 24.0/0. Average score (22.8)
Win/lose count 26.0/2.0. Average score (23.0)
Win/lose count 23.5/2.0. Average score (22.785714285714285)
Win/lose count 25.0/1.0. Average score (22.9375)
Win/lose count 15.0/7.0. Average score (21.27777777777778)
Win/lose count 21.0/2.0. Average score (21.05)
Final score: 21.05


The method with an exploration memory is achieving very good results and manages to find almost all the pieces of cheese. This will allow us to transform this reinforcement problem in a supervised learning problem by giving us the opportunity to generate winning games.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***

In [24]:
def save_for_training(state,action):
    state_string = ''
    w,h,d = state.shape
    for i in range(w):
        for j in range(h):
            for k in range(d):
                state_string += str(state[i,j,k]) + '/'
            state_string = state_string[:-1]
            state_string += '_'
        state_string = state_string[:-1]
        state_string += ' '
    state_string = state_string[:-1]
    with open('states.txt','a') as f:
        f.write(state_string + '\n')
        f.close()
    action_string = str(action)
    with open('actions.txt','a') as f:
        f.write(action_string + '\n')
        f.close()

def generate_GT(trained_agent,env,epoch,prefix='mimick'):
    with open('states.txt','w') as f:
        f.close()
    with open('actions.txt','w') as f:
        f.close()
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
        # The agent performs an action
            action = agent.act(state, train = False)
            save_for_training(state,action)
            prev_state = state
            state, reward, game_over = env.act(action)

        # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epoch)) 
    
generate_GT(agent, env, 5)

Win/lose count 23.5/1.0. Average score (22.5)
Win/lose count 24.5/2.0. Average score (22.5)
Win/lose count 25.5/1.0. Average score (23.166666666666668)
Win/lose count 27.0/0. Average score (24.125)
Win/lose count 22.0/1.0. Average score (23.5)
Final score: 23.5


Using the previous solver we generate and save some training data for the supervised learning. Each state is saved in a file named 'states.txt' and the corresponding ground, truth correct action to take, is saved in 'actions.txt'.

In [25]:
def load_training_data():
    states = []
    actions = []
    with open('states.txt','r') as f:
        string_states = f.read()
        f.close()
    string_states = string_states.splitlines()[:-1]
    for string in string_states:
        state = []
        for a in string.split(' '):
            tmp1 = []
            for b in a.split('_'):
                tmp2 = []
                for c in b.split('/'):
                    if c != '':
                        tmp2.append(float(c))
                tmp1.append(tmp2)
            state.append(tmp1)
        states.append(state)
    with open('actions.txt','r') as f:
        string_actions = f.read()
        f.close()
    string_actions = string_actions.splitlines()[:-1]
    for a in string_actions:
        tmp = np.zeros(4)
        tmp[int(a)] = 1
        actions.append(tmp)
    states = np.array(states)
    actions = np.array(actions)
    return states,actions

States, Actions = load_training_data()

We load the training data and store it in the variables States and Actions

In [26]:
class DQN_mimick(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_mimick, self).__init__(*args,**kwargs)
        
        input_layer = Input(shape=(5,5,self.n_state))
        x = Conv2D(32, (2, 2))(input_layer)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Conv2D(32, (2, 2))(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Flatten()(x)
        x = Dense(64)(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        output_layer = Dense(self.n_action)(x)
        model = Model(inputs=input_layer, outputs=output_layer)
        
        print(model.summary())
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mae")
        self.model = model
    
    def train(self, data, ground_truth):
        l = self.model.train_on_batch(data, ground_truth)
        return l
        
        
agent_mimick = DQN_mimick(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32, n_state=3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 5, 5, 3)           0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4, 4, 32)          416       
_________________________________________________________________
batch_normalization_18 (Batc (None, 4, 4, 32)          128       
_________________________________________________________________
activation_18 (Activation)   (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 3, 32)          4128      
_________________________________________________________________
batch_normalization_19 (Batc (None, 3, 3, 32)          128       
_________________________________________________________________
activation_19 (Activation)   (None, 3, 3, 32)          0         
__________

We created an agent that will imitate the previous expert one.

In [29]:
def train_mimick(agent,data,ground_truth,epoch,prefix='mimick_'):
    for e in range(epoch):
        loss = 0
        indeces = np.arange(data.shape[0])
        np.random.shuffle(indeces)
        batch_size = 32
        for b_ind in range(int(data.shape[0]/batch_size) + 1):
            batch = indeces[b_ind * batch_size: min((b_ind + 1) * batch_size, data.shape[0])]
            batch_data = data[batch,:,:,:]
            batch_gt = ground_truth[batch]
            loss += agent.train(batch_data,batch_gt)
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} "
              .format(e, epoch, loss))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

train_mimick(agent_mimick, States, Actions, 100)

Epoch 000/100 | Loss 7.6306 
Epoch 001/100 | Loss 7.5313 
Epoch 002/100 | Loss 7.3523 
Epoch 003/100 | Loss 7.2407 
Epoch 004/100 | Loss 7.0640 
Epoch 005/100 | Loss 7.0702 
Epoch 006/100 | Loss 6.9625 
Epoch 007/100 | Loss 6.7933 
Epoch 008/100 | Loss 6.7886 
Epoch 009/100 | Loss 6.6184 
Epoch 010/100 | Loss 6.6474 
Epoch 011/100 | Loss 6.5059 
Epoch 012/100 | Loss 6.5851 
Epoch 013/100 | Loss 6.3497 
Epoch 014/100 | Loss 6.2145 
Epoch 015/100 | Loss 6.1126 
Epoch 016/100 | Loss 6.3734 
Epoch 017/100 | Loss 6.1527 
Epoch 018/100 | Loss 5.9120 
Epoch 019/100 | Loss 6.1262 
Epoch 020/100 | Loss 5.9495 
Epoch 021/100 | Loss 5.9796 
Epoch 022/100 | Loss 5.8635 
Epoch 023/100 | Loss 5.8084 
Epoch 024/100 | Loss 5.8407 
Epoch 025/100 | Loss 5.5833 
Epoch 026/100 | Loss 5.8042 
Epoch 027/100 | Loss 5.5735 
Epoch 028/100 | Loss 5.5506 
Epoch 029/100 | Loss 5.5122 
Epoch 030/100 | Loss 5.4105 
Epoch 031/100 | Loss 5.4271 
Epoch 032/100 | Loss 5.5604 
Epoch 033/100 | Loss 5.1969 
Epoch 034/100 

In [30]:
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
test(agent_mimick,env,epochs_test,prefix='mimick_test_explore')
HTML(display_videos('mimick_test_explore0.mp4'))

Win/lose count 24.0/8.0. Average score (16.0)
Win/lose count 18.0/3.0. Average score (15.5)
Win/lose count 16.0/1.0. Average score (15.333333333333334)
Win/lose count 25.0/10.0. Average score (15.25)
Win/lose count 21.5/4.0. Average score (15.7)
Win/lose count 18.5/4.0. Average score (15.5)
Win/lose count 17.5/4.0. Average score (15.214285714285714)
Win/lose count 22.5/7.0. Average score (15.25)
Win/lose count 24.5/5.0. Average score (15.722222222222221)
Win/lose count 16.0/9.0. Average score (14.85)
Final score: 14.85


We observe that even with a small amount of data and far less game sessions, the algorithm captured most of the strategy and achieves close results in terms of performances.